# Урок 14. Transfer learning
## Домашнее задание.

1. Взять данные из [Kaggle](https://www.kaggle.com/datasets/mrapplexz/bashim-quotes) и обучить модель GPT для генерации своих цитат
2. Взять новостные данные из [GitHub](https://github.com/natasha/corus) `load_lenta2` (нам понадобиться сам текст и заголовок) и обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.1 MB/s eta 0:00:0

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset, load_metric

In [3]:
corpus = load_dataset('merionum/ru_paraphraser')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [20]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"
batch_size = 4

In [21]:
from transformers import AutoTokenizer, BertTokenizerFast

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [22]:
label_list = sorted(set(corpus['train']['class']))
label_list 

['-1', '0', '1']

In [23]:
labels2id = { key:id for id, key in enumerate(label_list)}
labels2id 

{'-1': 0, '0': 1, '1': 2}

In [24]:
id2labels = { id:key for id, key in enumerate(label_list)}
id2labels

{0: '-1', 1: '0', 2: '1'}

In [25]:
def tokenize_and_align_labels(tokenizer, labels2id):    
    def tokenize_and_align_labels_(examples):
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)
        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]
        return tokenized_inputs
    return tokenize_and_align_labels_
'''
Этот фрагмент кода определяет функцию 'tokenize_and_align_labels' для токенизации и выравнивания меток. 
Он принимает в качестве входных данных сопоставление tokenizer и labels2id и возвращает функцию 'tokenizeandalignlabels'. 
Эта функция принимает словарь примеров в качестве входных данных и токенизирует текст в полях 'text1' и 'text_2'. 
Затем он преобразует метки классов в поле 'class' в соответствующие идентификаторы с помощью сопоставления labels2id. 
Наконец, он возвращает словарь, содержащий токенизированные входные данные и метки.
'''

"\nЭтот фрагмент кода определяет функцию 'tokenize_and_align_labels' для токенизации и выравнивания меток. \nОн принимает в качестве входных данных сопоставление tokenizer и labels2id и возвращает функцию 'tokenizeandalignlabels'. \nЭта функция принимает словарь примеров в качестве входных данных и токенизирует текст в полях 'text1' и 'text_2'. \nЗатем он преобразует метки классов в поле 'class' в соответствующие идентификаторы с помощью сопоставления labels2id. \nНаконец, он возвращает словарь, содержащий токенизированные входные данные и метки.\n"

In [26]:
tokenized_input = tokenize_and_align_labels(tokenizer, labels2id)(corpus['train'][20:24])
tokenized_input

{'input_ids': [[0, 18449, 1068, 1951, 169066, 59, 42136, 59392, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2, 2, 6, 144048, 145158, 184643, 18449, 28196, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2], [0, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 7329, 155333, 24863, 5, 2, 2, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 534, 11591, 149673, 5, 2], [0, 417, 14100, 695, 1200, 49, 30037, 1269, 326, 3699, 108854, 1857, 56869, 90229, 546, 209, 9561, 5, 2, 2, 417, 68642, 69, 53177, 29, 108854, 1857, 47745, 49, 14100, 695, 1200, 90229, 546, 209, 9561, 5, 2], [0, 72085, 138109, 87830, 29, 23262, 2294, 90569, 11974, 24744, 59, 161679, 8568, 5, 2, 2, 69242, 64393, 41976, 4560, 1560, 182076, 29, 29230, 23262, 49, 72085, 138109, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [27]:
tokenized_input = tokenize_and_align_labels(tokenizer, labels2id)(corpus['train'][0:5])
tokenized_input

{'input_ids': [[0, 901, 128603, 17575, 92173, 1047, 58191, 12919, 29, 129, 53222, 129, 208167, 135, 70138, 7574, 7613, 5, 2, 2, 58083, 8441, 9072, 194266, 1117, 58191, 12919, 129, 32455, 546, 4132, 1258, 135, 70138, 7574, 7613, 5, 2], [0, 67694, 183014, 244, 29, 90089, 5930, 17377, 49, 185526, 95189, 102069, 1117, 5, 2, 2, 68815, 46665, 25195, 22051, 1270, 90089, 5930, 13617, 183014, 244, 49, 185526, 218476, 222, 5, 2], [0, 20224, 211356, 59, 49, 35308, 216019, 103, 42662, 49, 36110, 226, 146025, 171410, 5, 2, 2, 72504, 640, 211356, 59, 27304, 21397, 32942, 49, 15404, 49, 15115, 216019, 103, 42662, 5, 2], [0, 19493, 957, 125867, 424, 113899, 89, 203500, 90661, 67754, 13962, 16231, 197, 238382, 29, 94364, 103, 5, 2, 2, 17667, 7330, 227, 1435, 8113, 1560, 811, 27449, 13058, 203500, 424, 1080, 79734, 1857, 113899, 89, 5, 2], [0, 417, 165319, 424, 113899, 89, 165616, 6878, 116, 62705, 59, 1435, 8113, 1560, 135, 203500, 1281, 29, 18999, 1882, 5, 2, 2, 17667, 7330, 227, 1435, 8113, 1560, 811

In [28]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
print(tokens)

['<s>', '▁По', 'лицей', 'ским', '▁разреш', 'ат', '▁стрел', 'ять', '▁на', '▁по', 'ражение', '▁по', '▁гражданам', '▁с', '▁травм', 'ати', 'кой', '.', '</s>', '</s>', '▁Поли', 'ции', '▁могут', '▁разреши', 'ть', '▁стрел', 'ять', '▁по', '▁ху', 'ли', 'ган', 'ам', '▁с', '▁травм', 'ати', 'кой', '.', '</s>']


In [29]:
corpus['train'][0]

{'id': '1',
 'id_1': '201',
 'id_2': '8159',
 'text_1': 'Полицейским разрешат стрелять на поражение по гражданам с травматикой.',
 'text_2': 'Полиции могут разрешить стрелять по хулиганам с травматикой.',
 'class': '0'}

In [30]:
tokenized_datasets = corpus.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)

Map:   0%|          | 0/7227 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [31]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1924
    })
})

In [32]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))

model.config.id2label = id2labels 
model.config.label2id = labels2id 



In [34]:
from datasets import load_metric

metric = load_metric("accuracy")

<ipython-input-34-350fcae91df9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [35]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels)

In [36]:
args = TrainingArguments(
    "paraphras",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.05,
    save_strategy='no',
    report_to='none',
)

In [37]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [38]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_1, id_2, text_2, id, id_1, class. If text_1, id_2, text_2, id, id_1, class are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7227
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1807
  Number of trainable parameters = 559893507
You're using a XLMRobertaTokenizerFast tokenizer. Please note th

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_1, id_2, text_2, id, id_1, class. If text_1, id_2, text_2, id, id_1, class are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1807, training_loss=0.7419647571456056, metrics={'train_runtime': 705.9224, 'train_samples_per_second': 10.238, 'train_steps_per_second': 2.56, 'total_flos': 545117983134090.0, 'train_loss': 0.7419647571456056, 'epoch': 1.0})

In [39]:
example = tokenized_datasets["test"][40:80]

In [40]:
import torch 
tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')

with torch.no_grad():
    outputs = model(**tokens)

predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]

In [41]:
df_example =pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'], 'class':example['class'], 'predict':classes})
df_example

text_1  \
0   Владимир Путин освободил от должности почти 20...   
1   Источник: Якунин озвучил свою месячную зарплат...   
2            Apple получила патент на социальную сеть   
3   Подвесной потолок обрушился в аэропорту Стамбу...   
4        Россия представляет принципиально новый танк   
5   Генсек ООН и премьер Украины обсудили минские ...   
6   Результаты выборов в Великобритании укрепили фунт   
7   Николич: Сербия не говорила о нежелании строит...   
8   СМИ сообщают об обысках в Минкульте по делу о ...   
9    Любляна отпразднует День Победы вместе с Москвой   
10  Премьер Словакии прибыл в Москву для участия в...   
11  Кремль: Порошенко не был приглашен в Москву на...   
12                «Прогресс» сгорел над Тихим океаном   
13  Дворнику на Римского-Корсакова подкинули трехм...   
14  СМИ: украинский боксер разгневал своих фанатов...   
15  Самолеты коалиции начали бомбардировку йеменск...   
16  Греция подтвердила США интерес к газопроводу о...   
17  Хаджииоанну: США могут предложить Греции лишь ...   
18     Разговор о роли России во Второй мировой войне   
19  Путеводитель по Дню Победы: как провести 9 мая...   
20  Коморовский: отсутствие Меркель в Москве 9 мая...   
21  Аваков уволил 15 руководителей МВД Украины в р...   
22  Бывший ядерщик обвинен в США за попытку кражи ...   
23  Бывший ядерщик обвинен в США за попытку кражи ...   
24  Бывший ядерщик обвинен в США за попытку кражи ...   
25  Четыре человека погибли при крушении легкомото...   
26  Легкомоторный самолет упал в штате Вашингтон, ...   
27  КНДР заявила о проведении подводного запуска б...   
28  КНДР провела испытательный подводный пуск балл...   
29  КНДР заявила о проведении подводного запуска б...   
30  Путеводитель по Дню Победы: как провести 9 мая...   
31  Облака над Москвой в День Победы разгонят девя...   
32                        День Победы в режиме онлайн   
33              Около 40 китов погибли у берегов Чили   
34  Принц Чарльз надеется, что внучка будет заботи...   
35  Апакан призвал стороны украинского конфликта с...   
36  Пентагон: США считают вероятным приобретение К...   
37  Вице-президент Гватемалы ушла в отставку из-за...   
38            СМИ: скончался бас-гитарист группы ABBA   
39            СМИ: скончался бас-гитарист группы ABBA   

                                               text_2 class predict  
0   Путин снял с должностей более 20 руководителей...     0       1  
1                 Якунин назвал размер своей зарплаты     0       1  
2        Apple получила патент на собственную соцсеть     1       1  
3   Четверо пострадали при обрушении подвесного по...     1       1  
4         Вашингтон пригрозил России новыми санкциями    -1       1  
5   Генсек ООН и премьер-министр Украины обсудили ...     1       1  
6   Стали известны окончательные результаты выборо...    -1       1  
7   Президент Сербии опроверг свои слова об отказе...     0       1  
8   Минкультуры отчиталось перед следствием по дел...    -1       1  
9   Путеводитель по Дню Победы: как провести 9 мая...    -1       1  
10  Путеводитель по Дню Победы: как провести 9 мая...    -1       1  
11  Путеводитель по Дню Победы: как провести 9 мая...    -1       1  
12  Путин: кооперация с Китаем поспособствует прит...    -1       1  
13   Светлана Медведева награждена высшим орденом РПЦ    -1       1  
14  Украинский боксер удалил фотографию с георгиев...    -1       1  
15  Самолеты Арабской коалиции начали бомбить йеме...     1       1  
16  США предложили Греции отказаться от участия в ...    -1       1  
17  США предложили Греции отказаться от участия в ...    -1       1  
18  Консервативная партия получила большинство в б...    -1       0  
19  Лукашенко назвал надуманной проблему своего от...    -1      -1  
20  Лукашенко назвал надуманной проблему своего от...    -1       1  
21  Аваков уволил 15 региональных руководителей МВ...     0       1  
22  Бывший ядерщик обвинен в США за попытку кражи ...     0       1  
23  Бывший американский чинов